# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение Маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хранить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

### Считываем данные

Будем использовать новостной корпус Ленты ру и немного (около 300 текстов) статей The Village за 20-ый год.

In [3]:
lenta_ru = open('lenta.txt', encoding='utf-8').read()
the_village = open('village.txt', encoding='utf-8').read()

In [4]:
news_corpus = ''.join([the_village, lenta_ru])

### Обрабатываем данные

In [5]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from nltk.tokenize import sent_tokenize

import numpy as np
from scipy.sparse import lil_matrix

from tqdm import tqdm
from collections import Counter

In [6]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20]
    return normalized_text

In [7]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0, len(tokens) - n + 1):
        ngrams.append(' '.join(tokens[i: i + n]))
    return ngrams

In [8]:
sentences_news = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in tqdm(sent_tokenize(news_corpus))]

100%|████████████████████████████████████████████████████████████████████████| 103372/103372 [00:29<00:00, 3536.37it/s]


In [7]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

Матрица биграмы на слова

In [8]:
matrix_news_bi = lil_matrix((len(bigrams_news), 
                        len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

for ngram in trigrams_news:
    i = (ngram[::-1].index(' ') + 1) * -1
    bigram = ngram[:i]
    unigram = ngram[i + 1:]
    bi_ind = bigram2id_news[bigram]
    uni_ind = word2id_news[unigram]
    matrix_news_bi[bi_ind, uni_ind] = (trigrams_news[ngram] / bigrams_news[bigram])

### Функции генерации

Добавлено немного больших букв и точек, чтобы предложения выглядели симпатичнее

In [27]:
def generate_bi(matrix, id2word, id2bigram, bigram2id, start, n=100):
    text = []
    current_idx = bigram2id[start]
    upper = False
    
    for i in range(n):
        
        # получаем продолжение новой биграмы
        # если вероятность получить биграму слишком мала, ставим ... и выходим из цикла
        try:
            chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        except ValueError:
            text.append('...')
            break
        token = id2word[chosen]
        # создаем новую биграму
        new_token = id2bigram[current_idx].split()[1] + ' ' + token
        # добавляем только продолжение биграмы
        if upper:
            text.append(token.title())
            upper = False
        elif token == '<end>':
            text.extend(['.', token])
            new_token = '<start> <start>'
            upper = True
        else:
            text.append(token)
        # обновляем индекс биграмы
        current_idx = bigram2id[new_token]
    
    return ' '.join(text)

### Эксперименты

Примеры текстов для стартового токена и начала предложения

In [17]:
print('Как сообщает', generate_bi(matrix_news_bi, id2word_news, id2bigram_news,
                                  bigram2id_news, start='как сообщает', n=100).replace('<end>', '\n').replace('\n ', '\n'))

Как сообщает associated press 11 из 12 двигателей обсерватории работают нормально . 
Особое внимание на то законных оснований . 
Полные имена не разглашаются но уже в среду единственный регистратор доменных имен станет обычной практикой и не отвечал ни на один маникюр с покрытием у нас с мужем . 
Учтено было также объявлено что компания выступавшая и выступающая гарантом по части 3 статьи 46 федерального закона об охране подземных коммуникаций города сообщает радиостанция эхо москвы . 
Расход этих средств россии осталось выплатить около 230 миллионов абонентов сетей gsm . 
От удара подвесной топливный бак самолета получил повреждение произошло возгорание топлива в россии кто


In [18]:
print('Последние новости', generate_bi(matrix_news_bi, id2word_news, id2bigram_news,
                                  bigram2id_news, start='последние новости', n=100).replace('<end>', '\n').replace('\n ', '\n'))

Последние новости о своем выходе из тюрьмы . 
Цик рассмотрел также результаты дополнительной проверки проводки которая была сделана операция . 
Другой причиной катастрофы называлась акция итальянских диверсантов которые якобы планировали совершить теракты на железной дороге . 
Проще если воспринимать его не обнародовать имена иранских агентов поскольку их функции ограничены . 
Как сообщил риа новости со ссылкой на департамент правительственной информации рассеянные силы бандитов пытаются уйти в отставку с поста главы партии шинн фейн джерри адамс лидер наиболее радикально настроенного крыла ира шинн фейн уже заявил о своем уходе с российского рынка объясняются тем что сегодня нет человека который некогда считался самым богатым


In [21]:
print('В Москве', generate_bi(matrix_news_bi, id2word_news, id2bigram_news,
                                  bigram2id_news, start='в москве', n=100).replace('<end>', '\n').replace('\n ', '\n'))

В Москве потребление импортного пива снизилось с 35 до 50 рублей мясо — от 200 до 400 человек . 
Что же касается российских военных баз в афганистане михаила лиходеяпогибли 14 человек в марокко и что делает их недостоверными . 
Представители правительства заявили что корпорация обязательно будет участвовать и america online cnn netscape warner brothers sony music entertainment заставляет владельцев музыкальных магазинов продавать компакт-диски которые туристы очевидно слушали в дороге часов девять от встречи с и о том как манипулятивный бисексуальный школьник-ауткаст ко второму чтению в профильный комитет на доработку . 
Не могу комментировать поскольку был в ярости . 
Задержаны 26 человек поддерживавших


In [29]:
print(generate_bi(matrix_news_bi, id2word_news, id2bigram_news,
                  bigram2id_news, start='<start> <start>', n=100).replace('<end>', '\n').replace('\n ', '\n'))

умные преступления всегда совершают индивидуальности и каждой ступеньке империи микки — от 1 300 фунтов . 
По сведениям оперативного дежурного мвд абхазии виолетта чуева подрыв автомашины квалифицируется как особо тяжкое преступление . 
Нам не понадобится введения никакого чрезвычайного положения президент может распустить парламент и захватить его . 
Затем подельники садились в машину и уехали в вологду поздравить наших мам и там же где и предполагается что он пытался переправить в чечню . 
Соответствующий приказ подписал в среду приблизительно в полтора раза . 
Как сообщает риа новости ужесточен контроль за перевозками цветного лома на автотранспорте . 
В основном вице-премьеры иминистры которые


### Подсчёт перплексии

In [30]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [40]:
def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)
    
    return prob, len(tokens)


def compute_join_proba_markov_assumption(text, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>', '<start>'] + tokens + ['<end>']):
        i = (ngram[::-1].index(' ') + 1) * -1
        bigram = ngram[:i]
        if bigram in bigram_counts and ngram in bigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

In [31]:
dev = open('additional_sent.txt', encoding='utf-8').read()
dev = dev.split('\n')

In [32]:
phrase = 'В Москве потребление импортного пива снизилось с 35 до 50 рублей мясо — от 200 до 400 человек'

In [43]:
perplexity(*compute_join_proba_markov_assumption(phrase, bigrams_news, trigrams_news))

166371.05959946275

In [41]:
ps = []
for sent in dev:
    prob, N = compute_join_proba_markov_assumption(sent, bigrams_news, trigrams_news)
    if not N:
        continue
    ps.append(perplexity(prob, N))

In [51]:
np.mean(ps)

198176.28026769587

1. Сначала подсчитаем перплексию на одном из сгенерированных текстов

2. Затем среднюю перплексию на 25 отложенных предложениях.

Вывод: можно видеть, что результаты получаются примерно похожи.

### Эксперимент с матрицей биграмы на биграмы

In [9]:
bigrams_news = Counter()
fourgrams_news = Counter()

for sentence in sentences_news:
    bigrams_news.update(ngrammer(sentence))
    fourgrams_news.update(ngrammer(sentence, n=4))

In [10]:
matrix_news_bi = lil_matrix((len(bigrams_news), 
                        len(bigrams_news)))

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

for ngram in fourgrams_news:
    bigram1 = ' '.join(ngram.split()[:2])
    bigram2 = ' '.join(ngram.split()[2:])
    matrix_news_bi[bigram2id_news[bigram1], 
                   bigram2id_news[bigram2]] = (fourgrams_news[ngram] / bigrams_news[bigram1])

In [18]:
def generate_bi(matrix, id2word, word2id, start, n=100):
    text = []
    current_idx = word2id[start]
    upper = False
    
    for i in range(n):
        
        try:
            chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        except ValueError:
            text.append('...')
            break
        
        if upper:
            text.append(id2word[chosen].title())
            upper = False
        elif '<end>' in id2word[chosen]:
            text.extend([id2word[chosen], '.'])
            chosen = word2id['<start> <start>']
            upper = True
        else:
            text.append(id2word[chosen]) 
            
        current_idx = chosen
    
    return ' '.join(text)

In [19]:
print(generate_bi(matrix_news_bi, id2bigram_news, bigram2id_news, start='<start> <start>', n=100).replace('<end>', ''))

новейшие продукты современной электроники он с апреля этого года когда начал испытывать трудности с хлебом существуют по всей стране с частным визитом ...


In [21]:
print('Как сообщает',
      generate_bi(matrix_news_bi, id2bigram_news, bigram2id_news, start='как сообщает', n=100).replace('<end>', ''))

Как сообщает bbc в субботу во время телефонного разговора с шахтерами она даже спускалась в шахту несмотря на многочисленные требования собравшихся никто из правления ао к ним не съездишь а жаль  . За Прошлый год не более 60-70 километров  . Однако Многие сотрудники оон покидают афганистан из-за серии нападений на инкассаторов  . — Ты же я недавно переехавший в москву ...


In [32]:
print('По данным',
      generate_bi(matrix_news_bi, id2bigram_news, bigram2id_news, start='по данным', n=100).replace('<end>', ''))

По данным наших источников именно боевики заставляют мирных жителей возвращаться в грозный ...


In [33]:
print('По данным',
      generate_bi(matrix_news_bi, id2bigram_news, bigram2id_news, start='по данным', n=100).replace('<end>', ''))

По данным эха москвы с раннего утра привычно выстроились с плакатами у дома правительства в зданиях судов нет рукопись романа тихий дон ...


С матрицей биграмы на биграмы тексты получаются более связными, но из-за учитывания только частотности модель может резко менять тему. Также без сглаживания и обработки незнакомых слов модель быстро прекращает генерацию

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

- создать систему "открытого словаря", то есть иначе - заменять незнакомые слова **тестовой выборки** на какой-нибудь тег, например, \<unk>. 

Как в этом случае мы зададим вероятность данного тега? - 1) задать словарь токенов на обучающей выборке, 2) нормализовать тестовые данные по фиксированному словарю обучающей выборки, заменив отсутствующие слова тегом \<unk>, 3) обновить вероятности с учетом данного тега

- заменять тегом \<unk> случайные слова в обучающей выборке

2. Что такое сглаживание (smoothing)?

Сглаживание - это техника, которая нужна, чтобы модель также распознавала уже знакомые слова, но в незнакомых контекстах. Есть разные виды сглаживания `Laplace (add-one) smoothing, add-k smoothing, stupid backoff, and Kneser-Ney smoothing.`

В простейшем случае можно добавить 1 к частотности каждой нграмы. Тогда нграмы, ни разу не встретившиеся в корпусе, теперь будут иметь частотность 1.